# Deploy Model Online

In [50]:
import pickle
import boto3

In [128]:
# Upload the model to aws bucket

s3 = boto3.client("s3")
s3.upload_file("life_allocation.pkl", "gamification-ee16ff5ab7", "life_allocation.pkl")

## Write Predictor Class

In [127]:
%%writefile predictor.py

import os
import pickle
import numpy as np
import json

labels = ["False", "True"]


# home purchase model
class PythonPredictor:
    def __init__(self, config):
        """
        if os.environ.get("AWS_ACCESS_KEY_ID"):
            s3 = boto3.client("s3")  # client will use your credentials if available
        else:
            s3 = boto3.client("s3", config=Config(signature_version=UNSIGNED))  # anonymous client

        s3.download_file(config["bucket"], config["key"], "/tmp/model.pkl")
        """
        self.model = pickle.load(open("home_purchase.pkl", "rb"))

    def predict(self, payload):
   
        proba = self.model.predict_proba([payload['data']])[0]
        result = [float(val) for val in proba]
        return json.dumps({'prob':result, 'label':int(np.argmax(result))})

Overwriting predictor.py


In [129]:
%%writefile predictor.py

import os
import pickle
import numpy as np
import json

labels = ["False", "True"]

# life relocation model
class PythonPredictor:
    def __init__(self, config):
        """
        if os.environ.get("AWS_ACCESS_KEY_ID"):
            s3 = boto3.client("s3")  # client will use your credentials if available
        else:
            s3 = boto3.client("s3", config=Config(signature_version=UNSIGNED))  # anonymous client

        s3.download_file(config["bucket"], config["key"], "/tmp/model.pkl")
        """
        self.model = pickle.load(open("life_allocation.pkl", "rb"))

    def predict(self, payload):
   
        proba = self.model.predict_proba([payload['data']])[0]
        result = [float(val) for val in proba]
        return json.dumps({'prob':result, 'label':int(np.argmax(result))})

Overwriting predictor.py


In [125]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('../../assets/dataset/relocation_db.csv', index_col=None)
df.shape
df.head()

y = df.loc[:,'life_allocation']
X = df.loc[:,:'ACNT_SEC_CLOSE_*']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [126]:
# test predictor
p = {"data": [int(val) for val in X_test.iloc[i].values]}
rf = PythonPredictor(None)
rf.predict(p)

'{"prob": [0.6698397851987394, 0.33016021480126057], "label": 0}'

## Deployment Configuration

In [53]:
%%writefile cortex.yaml
- name: home-purchase-model
  kind: RealtimeAPI
  predictor:
    type: python
    path: predictor.py
    config:
      bucket: gamification-ee16ff5ab7
      key: dev/home_purchase.pkl

Overwriting cortex.yaml


In [130]:
%%writefile cortex.yaml
- name: life-relocation-model
  kind: RealtimeAPI
  predictor:
    type: python
    path: predictor.py
    config:
      bucket: gamification-ee16ff5ab7
      key: dev/life_allocation.pkl

Overwriting cortex.yaml


## Deployment Requirements

In [19]:
%%writefile requirements.txt
pandas
scikit-learn
lightgbm

Writing requirements.txt


## Deploy Model
deploy the model in terminal

In [12]:
# !cortex deploy
# !cortex get 
# !cortex get recommender

## Query Deployed Model

### 1. Request

In [133]:
import requests 

##
# query home purchase model


df = pd.read_csv('../../assets/dataset/home_db.csv', index_col=None)
y = df.loc[:,'home_purchase']
X = df.loc[:,:'ACNT_SEC_CLOSE_*']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# api-endpoint 
#URL = "http://localhost:8888"
URL = "https://rqg11l7hg2.execute-api.us-west-1.amazonaws.com/home-purchase-model"

# send some queries
headers = {"Content-Type":"application/json"}
for i in range(10):
    p = {"data": [int(val) for val in X_test.iloc[i].values]}
    r = requests.post(url = URL, data = json.dumps(p), headers=headers)  
    print(r.json())

{'prob': [0.21531037233562156, 0.7846896276643784], 'label': 1}
{'prob': [0.21531037233562156, 0.7846896276643784], 'label': 1}
{'prob': [0.21531037233562156, 0.7846896276643784], 'label': 1}
{'prob': [0.6863351715635667, 0.3136648284364333], 'label': 0}
{'prob': [0.7790521441546707, 0.22094785584532922], 'label': 0}
{'prob': [0.21531037233562156, 0.7846896276643784], 'label': 1}
{'prob': [0.21531037233562156, 0.7846896276643784], 'label': 1}
{'prob': [0.30659267244415445, 0.6934073275558456], 'label': 1}
{'prob': [0.810935756318447, 0.18906424368155297], 'label': 0}
{'prob': [0.5584180618941934, 0.4415819381058066], 'label': 0}


In [134]:
import requests 

##
# query life relocation model


df = pd.read_csv('../../assets/dataset/relocation_db.csv', index_col=None)
df.shape
df.head()

y = df.loc[:,'life_allocation']
X = df.loc[:,:'ACNT_SEC_CLOSE_*']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# api-endpoint 
URL = "https://rqg11l7hg2.execute-api.us-west-1.amazonaws.com/life-relocation-model"

# send some queries
headers = {"Content-Type":"application/json"}
for i in range(10):
    p = {"data": [int(val) for val in X_test.iloc[i].values]}
    r = requests.post(url = URL, data = json.dumps(p), headers=headers)  
    print(r.json())

{'prob': [0.18250801831122854, 0.8174919816887715], 'label': 1}
{'prob': [0.18250801831122854, 0.8174919816887715], 'label': 1}
{'prob': [0.18250801831122854, 0.8174919816887715], 'label': 1}
{'prob': [0.18250801831122854, 0.8174919816887715], 'label': 1}
{'prob': [0.18250801831122854, 0.8174919816887715], 'label': 1}
{'prob': [0.18250801831122854, 0.8174919816887715], 'label': 1}
{'prob': [0.18250801831122854, 0.8174919816887715], 'label': 1}
{'prob': [0.18250801831122854, 0.8174919816887715], 'label': 1}
{'prob': [0.6698397851987394, 0.33016021480126057], 'label': 0}
{'prob': [0.6698397851987394, 0.33016021480126057], 'label': 0}


### 2. curl

In [17]:
payload

{'data': [1968, 4, 2, 0, 0, 0, 1, 0, 1, 0]}

In [18]:
%%writefile sample.json
{"data": [1968, 4, 2, 0, 0, 0, 1, 0, 1, 0]}

Overwriting sample.json


In [30]:
!curl http://localhost:8888 -X POST -H "Content-Type: application/json" -d @sample.json

False

In [55]:
!curl https://t0erni03x7.execute-api.us-west-1.amazonaws.com/gamification-model -X POST -H "Content-Type: application/json" -d @sample.json

True